En este archivo puedes escribir lo que estimes conveniente. Te recomendamos detallar tu solución y todas las suposiciones que estás considerando. Aquí puedes ejecutar las funciones que definiste en los otros archivos de la carpeta src, medir el tiempo, memoria, etc.

Para resolver este challenge hice uso de principalmente de dos herramientas para el procesamiento de datos: Pandas y PySpark. Ambas, ampliamente utilizadas en el ámbito de la ingeniería de datos.

Pandas por su parte es ideal para conjuntos de datos de tamaño moderado que caben cómodamente en la memoria de un solo nodo. Si bien su tiempo de ejecución es más lento que Pyspark, tiene una gran capacidad para trabajar con datos estructurados de forma intuitiva y realizar tareas de limpieza, transformación y preparación de datos.

Por otro lado, PySpark es ideal para trabajar con conjuntos de datos de gran escala. Gracias a su arquitectura distribuida aprovecha todos los nodos y si bien esto hace que consuma más memoria, el tiempo de ejecución es significativamente más bajo que herramientas como Pandas. Permite escalar horizontalmente y tiene gran tolerancia a fallos

Por todo esto decidí usar Pandas cuando el tamaño de los datos era manejable en memoria y las operaciones requeridas podían realizarse eficientemente en un solo nodo. Recurrí a PySpark cuando necesitábamos procesar los datos de manera distribuida para obtener resultados en un tiempo mucho menor.

In [ ]:
file_path = "farmers-protest-tweets-2021-2-4.json"

In [ ]:
import os
from q1_memory import *
from q2_memory import *
from q3_memory import *
from q1_time import *
from q2_time import *
from q3_time import *

In [ ]:
base_path = os.getcwd().replace('\\', '/')
file_path = "farmers-protest-tweets-2021-2-4.json"

In [ ]:
r_q1_memory = q1_memory(f'{base_path}/{file_path}')
print(r_q1_memory)

In [ ]:
r_q1_time = q1_time(f'{base_path}/{file_path}')
print(r_q1_time)

In [ ]:
r_q2_memory = q2_memory(f'{base_path}/{file_path}')
print(r_q2_memory)

In [ ]:
r_q2_time = q2_time(f'{base_path}/{file_path}')
print(r_q2_time)

In [ ]:
r_q3_memory = q3_memory(f'{base_path}/{file_path}')
print(r_q3_memory)

In [ ]:
r_q3_time = q3_time(f'{base_path}/{file_path}')
print(r_q3_time)